# Landmarks Extraction

In [10]:
DATA_DIR = r"/home/yiran.lu@Digital-Grenoble.local/Documents/PushUpData"
model_path = '/home/yiran.lu@Digital-Grenoble.local/Downloads/pose_landmarker_heavy.task'

In [11]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

## Load the Model

In [12]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)

landmarker = PoseLandmarker.create_from_options(options)

I0000 00:00:1765893215.973555   27385 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765893215.977425   28426 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.2), renderer: Mesa Intel(R) HD Graphics 530 (SKL GT2)
W0000 00:00:1765893216.081197   28429 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765893216.239228   28433 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Extract the Pose Landmarks from Images

In [13]:
# bas
from glob import glob
from tqdm import tqdm

bas_pose = []

for filename in tqdm(glob(f'{DATA_DIR}/*/bas/*.*')):
    mp_image = mp.Image.create_from_file(filename)
    pose_landmarker_result = landmarker.detect(mp_image)
    bas_pose.append(pose_landmarker_result)

100%|██████████| 275/275 [00:34<00:00,  8.03it/s]


In [14]:
# haut
from glob import glob

haut_pose = []

for filename in tqdm(glob(f'{DATA_DIR}/*/haut/*.*')):
    mp_image = mp.Image.create_from_file(filename)
    pose_landmarker_result = landmarker.detect(mp_image)
    haut_pose.append(pose_landmarker_result)

  0%|          | 0/270 [00:00<?, ?it/s]

100%|██████████| 270/270 [00:30<00:00,  8.89it/s]


In [15]:
import pickle

with open("bas_pose.pickle", "wb") as file:
    pickle.dump(bas_pose, file)

with open("haut_pose.pickle", "wb") as file:
    pickle.dump(haut_pose, file)

## Utility Function: Draw the Landmarks

In [7]:
import numpy as np
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [16]:
# mp_image = mp.Image.create_from_file(f"{DATA_DIR}/bas/captured_10003.png")
# pose_landmarker_result = landmarker.detect(mp_image)

# annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), pose_landmarker_result)

In [17]:
# from matplotlib import pyplot as plt
# plt.imshow(annotated_image)